# Assignment2 - Supervised Learning flow

# Part 1(a) Student details:
* Please write the First-Name, First letter of Last-Name and last 4 digits of the i.d. for each student. 

In [ ]:
# stundent details:
# Michael O J 5140
# Agam M 6895
# Ofek L 8567

## Part 1(b) - Chat-GPT/other AI-agent/other assistance used:
* If you changed the prompt until you got a satisfying answer, please add all versions
* don't delete "pre" tags, so new-line is supported
* double click the following markdown cell to change
* press shift+enter to view
* Add information:

    #### Add information in this Markdown cell (double click to change, shift-enter to view)
<pre>   
AI agent name: Coplilot-{Gpt, Claude, etc...}
Goal: Help us write proper code, prevent errors and to understand the pipeline better.
Propmpt1:
    "Fix the highlighted code"
Propmpt2:
    "Why do i get this error?"
Propmpt3:
    "Explain hyperparameters and their importance in ML"


AI agent name 2:
Goal:
Propmpt1:

Propmpt2:

Propmpt3:


Other assistanse:
</pre>

    ## Part 1(c) - Learning Problem and dataset explaination.
* Please explain in one paragraph
* don't delete "pre" tags, so new-line is supported
* double click the following markdown cell to change
* press shift+enter to view
* Add explaining text:

    #### Add information in this Markdown cell (double click to change, shift-enter to view)
<pre>
The learning problem involves predicting the median house value (`MedHouseVal`) based on various features such as the average number of rooms (`AveRooms`), population size (`Population`), and other housing-related attributes. The dataset used for this task is a housing dataset split into training (`housing_train.csv`) and test (`housing_test.csv`) sets. The training set is used to build and optimize machine learning models, while the test set is used to evaluate the model's performance. The goal is to identify the best combination of feature engineering techniques and algorithms to achieve accurate predictions of house prices.
</pre>

## Part 2 - Initial Preparations 
You could add as many code cells as needed

In [ ]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.pipeline import Pipeline

In [ ]:
df_train = pd.read_csv("housing_train.csv")
df_test = pd.read_csv("housing_test.csv")

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
# data cleansing
def clean_data(df):
    for column in df.columns:
        missing_percentage = df[column].isnull().mean()
        if missing_percentage < 0.1:
            df[column].fillna(df[column].median(), inplace=True)
        else:
            df.dropna(subset=[column], inplace=True)

    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True)
    return df

def remove_outliers(df, column, threshold=3):
    mean = df[column].mean()
    std_dev = df[column].std()
    lower_bound = mean - threshold * std_dev
    upper_bound = mean + threshold * std_dev
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]


In [ ]:
df_train = clean_data(df_train)
df_train = remove_outliers(df_train, 'MedHouseVal')
df_train = remove_outliers(df_train, 'AveRooms')
df_train = remove_outliers(df_train, 'Population')

In [ ]:
figure, axes = plt.subplots(1, 3, figsize=(15, 5))

col1, col2, col3 = "AveRooms", "Population", "MedHouseVal"

axes[0].scatter(df_train[col1], df_train[col2])
axes[0].set_xlabel(col1)
axes[0].set_ylabel(col2)
axes[0].set_title(f'{col1} vs {col2}')

axes[1].scatter(df_train[col1], df_train[col3])
axes[1].set_xlabel(col1)
axes[1].set_ylabel(col3)
axes[1].set_title(f'{col1} vs {col3}')

axes[2].scatter(df_train[col2], df_train[col3])
axes[2].set_xlabel(col2)
axes[2].set_ylabel(col3)
axes[2].set_title(f'{col2} vs {col3}')

plt.tight_layout()
plt.show()

In [ ]:
figure , axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].hist(df_train[col1], bins=20, color='red', alpha=0.7)
axes[0].set_title(f'{col1} Histogram')

axes[1].hist(df_train[col2], bins=20, color='green', alpha=0.7)
axes[1].set_title(f'{col2} Histogram')

axes[2].hist(df_train[col3], bins=20, color='blue', alpha=0.7)
axes[2].set_title(f'{col3} Histogram')

plt.tight_layout()
plt.show()

In [ ]:
corr = df_train.corr(method = 'pearson')
corr

The purpose of the visualizations is to explore the relationships and distributions of key features in the dataset. Scatter plots are used to examine potential correlations between variables (such as average rooms, population, and median house value), while histograms provide insights into the distribution of each feature, helping to identify patterns, outliers, or skewness in the data.

## Part 3 - Experiments
You could add as many code cells as needed

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNet

# Feature engineering options
feature_options = [
    [],  # No transformation
    ['scale'],  # Standard scaling
    ['poly2'],  # Polynomial degree 2
    ['selectk'],  # Feature selection
    ['scale', 'poly2'],  # Scaling + Polynomial
    ['scale', 'selectk'],  # Scaling + Feature selection
    ['poly2', 'selectk'],  # Polynomial + Feature selection
    ['scale', 'poly2', 'selectk']  # All transformations
]

# Models and their hyperparameters
models = {
    'LinearRegression': {
        'model': LinearRegression(),
        'params': {}
    },
    'RandomForest': {
        'model': RandomForestRegressor(random_state=42),
        'params': {'model__n_estimators': [50, 100], 'model__max_depth': [3, 5, None]}
    },
    'Ridge': {
        'model': Ridge(),
        'params': {'model__alpha': [0.1, 1, 10]}
    },
    'NeuralNetwork': {
        'model': MLPRegressor(random_state=42, max_iter=500),
        'params': {'model__hidden_layer_sizes': [(50,), (100,)], 'model__activation': ['relu', 'tanh']}
    },
    'XGBoost': {
        'model': XGBRegressor(tree_method='hist', device='cuda', random_state=42),
        'params': {'model__n_estimators': [50, 100], 'model__max_depth': [3, 5, None]}
    }
}
results = []

X = df_train.drop('MedHouseVal', axis=1)
y = df_train['MedHouseVal']

In [ ]:
for feats in feature_options:
    start_time = time.time()
    steps = []
    feats_name = []
    if 'scale' in feats:
        steps.append(('scaler', StandardScaler()))
        feats_name.append('scale')
    if 'poly2' in feats:
        steps.append(('poly', PolynomialFeatures(degree=2, include_bias=False)))
        feats_name.append('poly2')
    if 'selectk' in feats:
        steps.append(('selectk', SelectKBest(score_func=f_regression, k=2)))
        feats_name.append('selectk')
    for model_name, model_dict in models.items():
        print(f"Running model: {model_name} with features: {feats_name}")

        pipe_steps = steps + [('model', model_dict['model'])]
        pipe = Pipeline(pipe_steps)
        param_grid = model_dict['params']

        grid = GridSearchCV(pipe, param_grid, cv=5, scoring=make_scorer(mean_squared_error, greater_is_better=False), n_jobs=-1)
        grid.fit(X, y)

        elapsed_time = time.time() - start_time
        results.append({
            'Feature Engineering': '+'.join(feats_name) if feats_name else 'none',
            'Model': model_name,
            'Best Params': grid.best_params_,
            'CV Mean MSE': -grid.best_score_,
            'R² Score':  grid.best_estimator_.score(X, y),
            'Elapsed Time (s)': elapsed_time
        })

results_df = pd.DataFrame(results)
results_df = results_df.sort_values('R² Score', ascending=False)
display(results_df)

## Part 4 - Training 
Use the best combination of feature engineering, model (algorithm and hyperparameters) from the experiment part (part 3)

In [ ]:
best_result = results_df.iloc[0]
best_features = best_result['Feature Engineering'].split('+') if best_result['Feature Engineering'] != 'none' else []
best_model_name = best_result['Model']
best_params = {k: v for k, v in best_result['Best Params'].items() if k != 'model__model'}

steps = []
if 'scale' in best_features:
    steps.append(('scaler', StandardScaler()))
if 'poly2' in best_features:
    steps.append(('poly', PolynomialFeatures(degree=2, include_bias=False)))
if 'selectk' in best_features:
    steps.append(('selectk', SelectKBest(score_func=f_regression, k=2)))

best_model = models[best_model_name]['model']
steps.append(('model', best_model))

pipeline = Pipeline(steps)

pipeline.set_params(**best_params)

pipeline.fit(X, y)

## Part 5 - Apply on test and show model performance estimation

In [ ]:
X_test = df_test.drop('MedHouseVal', axis=1)
y_test = df_test['MedHouseVal']

y_pred = pipeline.predict(X_test)

cv_scores = cross_val_score(pipeline, X_test, y_test, cv=5, scoring=make_scorer(mean_squared_error, greater_is_better=False))
mean_cv_mse = -cv_scores.mean()
score = pipeline.score(X_test, y_test)

print(f"Cross-Validation Mean MSE on Test Set: {mean_cv_mse}")
print(f"R² Score on Test Set: {score}")
print(f"Model Accuracy: {score*100:.2f}%")

pd.DataFrame({'Actual': y_test, 'Predicted': y_pred}).head()

In [ ]:
if hasattr(best_model, 'feature_importances_'):
    feature_importance = best_model.feature_importances_
    feature_names = X.columns
    importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})
    importance_df = importance_df.sort_values(by='Importance', ascending=False)

    print("Feature Importance:")
    display(importance_df)

    plt.figure(figsize=(10, 6))
    plt.barh(importance_df['Feature'], importance_df['Importance'], color='skyblue')
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.title('Feature Importance')
    plt.gca().invert_yaxis()
    plt.show()

residuals = y_test - y_pred

plt.figure(figsize=(10, 6))
plt.scatter(y_test, residuals, alpha=0.7, color='purple')
plt.axhline(y=0, color='red', linestyle='--')
plt.xlabel('Actual Values')
plt.ylabel('Residuals')
plt.title('Residual Analysis')
plt.show()